## check the contents in laptime dataset

base:   


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import ipdb; 

In [89]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#check for car12
def check_laptimedata(_laptime_data, check_carno = 12, check_event = 'Indy500-2018', train_ratio=0):
    """
    print contents of laptimedataset
    """
    
    run_ts = COL_RANK
    totalTSCnt = 0
    totalTSLen = 0        
    
    for _data in _laptime_data:
        _train = []
        _test = []

        if not events[_data[0]] == check_event:
            continue

        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        if train_len == 0:
            #use global train_len
            train_len = _train_len

        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10

        print(f'after ====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()

            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]

            # remove short ts
            totallen = rec.shape[1]

            totalTSCnt += 1
            totalTSLen += totallen

            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                

            carno = _data[1][rowid]
            #carid = global_carids[_data[1][rowid]]

            if carno == check_carno:
                #show content in rec

                #rank
                print('rank:\n',rec[COL_RANK,:] )

                #pitlaps
                pits = np.where(rec[COL_LAPSTATUS,:]==1)
                print('pits:\n', pits)
                
                #leaderPitCnt
                print('leaderPitCnt:\n', rec[COL_LEADER_PITCNT,:])
                
def print_feature(_laptime_data, check_feature):
    """
    print contents of laptimedataset
    """
    
    print('print feature:', decode_feature_mode(check_feature))
    
    run_ts = COL_RANK
    for _data in _laptime_data:

        print('events:', _data[0])
        
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()

            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            recx = rec[:, ~np.isnan(rec[run_ts,:])]
            # remove short ts
            totallen = recx.shape[1]
            
            nans, x= nan_helper(rec[check_feature,:])
            nan_count = np.sum(nans)             
            recz = rec[:, ~np.isnan(rec[check_feature,:])]

            carno = _data[1][rowid]
            #carid = global_carids[_data[1][rowid]]
            #leaderPitCnt
            print('car:%s\n'%carno, 'totallen:', totallen, 'feature len:', recz.shape[1])
            print(rec[check_feature,:])                
                
def print_leaderpitcnt(_laptime_data):                
    print_feature(_laptime_data, COL_LEADER_PITCNT)
                
def print_laptimedata(_laptime_data, check_cars = [12], check_laps = [0,200], 
                      check_features=[COL_RANK,COL_LAPSTATUS,COL_LEADER_PITCNT],
                      check_event = 'Indy500-2018', train_ratio=0):
    """
    print contents of laptimedataset
    """
    
    run_ts = COL_RANK
    
    for _data in _laptime_data:
        if not events[_data[0]] == check_event:
            continue

        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        if train_len == 0:
            #use global train_len
            train_len = _train_len

        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10

        print(f'check ====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        #output, rank, pits, leaderPitCnt
        total_feature_cnt = _data[2][0].shape[0]
        total_lap_cnt = check_laps[1] - check_laps[0] 
        if len(check_cars) == 0:
            #get all carnos
            check_cars = list(_data[1].values)
        total_car_cnt = len(check_cars)
        
        out_mat = np.zeros((total_car_cnt, total_lap_cnt, total_feature_cnt))
        print('out_mat shape:', out_mat.shape)
        carids = {carno:idx for idx, carno in enumerate(check_cars)}
        print('carids:', carids)
        
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()

            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]

            # remove short ts
            totallen = rec.shape[1]

            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                

            carno = _data[1][rowid]
            #carid = global_carids[_data[1][rowid]]

            if carno in carids:
                
                carid = carids[carno]
                
                for fid in check_features:
                    out_mat[carid, 0:total_lap_cnt, fid] = rec[fid,check_laps[0]:check_laps[1]]

                #show content in rec
                #pitlaps
                pits = np.where(rec[COL_LAPSTATUS,:]==1)
                print('car:', carno, ' pits:\n', pits)
                
                #rank
                print('rank:\n',rec[COL_RANK,check_laps[0]:check_laps[1]] )
                #leaderPitCnt
                print('leaderPitCnt:\n', rec[COL_LEADER_PITCNT,check_laps[0]:check_laps[1]])              

    #out_mat
    sel_mat = out_mat[:, :, np.array(check_features)]
    
    print(sel_mat)
    
    return sel_mat
    

In [91]:
#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
COL_CAUTION_LAPS_INSTINT=5
COL_LAPS_INSTINT= 6
COL_ELAPSED_TIME= 7
COL_LAP2NEXTPIT = 8
#_featureCnt = 9

# added new features
COL_LEADER_PITCNT = 9
COL_TOTAL_PITCNT = 10
COL_SHIFT_TRACKSTATUS = 11
COL_SHIFT_LAPSTATUS = 12
COL_SHIFT_LEADER_PITCNT = 13
COL_SHIFT_TOTAL_PITCNT = 14


FEATURE_STATUS = 2
FEATURE_PITAGE = 4
FEATURE_LEADER_PITCNT = 8
FEATURE_TOTAL_PITCNT = 16
FEATURE_SHIFT_TRACKSTATUS = 32
FEATURE_SHIFT_LAPSTATUS = 64
FEATURE_SHIFT_LEADER_PITCNT = 128
FEATURE_SHIFT_TOTAL_PITCNT  = 256

_feature2str= {
    FEATURE_STATUS : ("FEATURE_STATUS",'S'),
    FEATURE_PITAGE : ("FEATURE_PITAGE",'P'),
    FEATURE_LEADER_PITCNT : ("FEATURE_LEADER_PITCNT",'L'),
    FEATURE_TOTAL_PITCNT : ("FEATURE_TOTAL_PITCNT",'T'),
    FEATURE_SHIFT_TRACKSTATUS : ("FEATURE_SHIFT_TRACKSTATUS",'S'),
    FEATURE_SHIFT_LAPSTATUS : ("FEATURE_SHIFT_LAPSTATUS",'P'),
    FEATURE_SHIFT_LEADER_PITCNT : ("FEATURE_SHIFT_LEADER_PITCNT",'L'),
    FEATURE_SHIFT_TOTAL_PITCNT  : ("FEATURE_SHIFT_TOTAL_PITCNT"'T')
    }


#_feature_mode = FEATURE_STATUS
def decode_feature_mode(feature_mode):
    
    retstr = []
    short_ret = []
    for feature in _feature2str.keys():
        if test_flag(feature_mode, feature):
            retstr.append(_feature2str[feature][0])
            short_ret.append(_feature2str[feature][1])
        else:
            short_ret.append('0')

    print(' '.join(retstr))
    
    return ''.join(short_ret)

#inlap status = 
# 0 , no inlap
# 1 , set previous lap
# 2 , set the next lap
_inlap_status = 0

#
# featuremode in [FEATURE_STATUS, FEATURE_PITAGE]:
#
_feature_mode = FEATURE_LEADERPITCNT
_featureCnt = 9

#
# training parameters
#
freq = "1min"
_train_len = 40
prediction_length = 2

context_ratio = 0.
context_length =  40
contextlen = context_length

dataset='rank'
_run_ts = COL_RANK

_test_event = 'Indy500-2018'
year = '2018'

#
# string map
#
inlapstr = {0:'noinlap',1:'inlap',2:'outlap'}
featurestr = {FEATURE_STATUS:'nopitage',FEATURE_PITAGE:'pitage',FEATURE_LEADERPITCNT:'leaderpitcnt'}
weightstr = {True:'weighted',False:'noweighted'}
catestr = {True:'cate',False:'nocate'}

#
# input data parameters
#
years = ['2013','2014','2015','2016','2017','2018','2019']
events = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(events)}
dbid = f'Indy500_{years[0]}_{years[-1]}_v{_featureCnt}_p{_inlap_status}'
_dataset_id = '%s-%s'%(inlapstr[_inlap_status], featurestr[_feature_mode])

# standard output file names
LAPTIME_DATASET = f'laptime_rank_timediff_pit-oracle-{dbid}.pickle' 
STAGE_DATASET = f'stagedata-{dbid}.pickle' 
EVALUATION_RESULT_DF = f'evaluation_result_d{dataset}.csv'
LONG_FORECASTING_DFS = f'long_forecasting_dfs_d{dataset}.pickle'
FORECAST_FIGS_DIR = f'forecast-figs-d{dataset}/'  

In [32]:
outdir = 'test/'
outputRoot = outdir
os.makedirs(outdir, exist_ok=True)

_task_dir = f'{outdir}/'

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
dbname = _task_dir + f'gluontsdb-{dataset}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{_test_event}-indy-{year}.pickle'
laptimedb = _task_dir + f'gluontsdb-{dataset}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{_test_event}-indy-{year}-newlaptimedata.pickle'

print('Load Laptime Dataset:',laptimedb)
with open(_task_dir + LAPTIME_DATASET, 'rb') as f:
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 

print('Load New Laptime Dataset:',laptimedb)
with open(laptimedb, 'rb') as f:
    prepared_laptimedata = pickle.load(f, encoding='latin1') 

Load Laptime Dataset: test//gluontsdb-rank-oracle-noip-noeid-all-all-f1min-t2-rIndy500-2018-indy-2018-newlaptimedata.pickle
Load New Laptime Dataset: test//gluontsdb-rank-oracle-noip-noeid-all-all-f1min-t2-rIndy500-2018-indy-2018-newlaptimedata.pickle


In [45]:
mat = print_laptimedata(prepared_laptimedata, check_cars = [6], check_laps = [90,100] )

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 10, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 16. 12.  9.  5.  3.  2.  2.  2.  2.]
leaderPitCnt:
 [0. 2. 1. 2. 0. 0. 0. 0. 0. 0.]
[[[16.  0.  0.]
  [16.  0.  2.]
  [12.  0.  1.]
  [ 9.  0.  2.]
  [ 5.  0.  0.]
  [ 3.  0.  0.]
  [ 2.  0.  0.]
  [ 2.  0.  0.]
  [ 2.  0.  0.]
  [ 2.  0.  0.]]]


In [46]:
matall = print_laptimedata(prepared_laptimedata, check_cars = [6])

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 200, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 15. 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 15. 15. 13.  9. 25. 22. 17. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 16. 15. 15. 15. 15.
 14. 14. 13. 13. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 14. 22. 21. 21.
 21. 20. 18. 18. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 18. 18.
 16. 16. 12.  9.  5.  3.  2.  2.  2.  2.  2.  2.  2.  2.  2. 11. 18. 18.
 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 16. 16. 16. 16. 15. 14.
 13. 12. 10.  7.  4.  4.  3.  3.  2.  2. 16. 19. 19. 19. 17. 15. 15. 15.
 15. 15. 16. 16. 16. 16. 16. 16. 17. 15. 14. 14. 13. 13. 13. 13. 11. 15.
 15. 15. 15. 15. 15. 14. 14. 12. 11.  7.  6.  4.  4.  2.  2.  1.  1. 11.
 20. 20. 20. 20. 20. 20. 20. 19. 19. 19. 19. 19. 18. 18. 1

In [52]:
mat49 = print_laptimedata(prepared_laptimedata, check_cars = [6], check_laps = [25,35])

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 10, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 16. 16. 15. 15. 13.  9. 25. 22. 17.]
leaderPitCnt:
 [0. 0. 0. 0. 1. 3. 5. 2. 6. 1.]
[[[16.  0.  0.]
  [16.  0.  0.]
  [16.  0.  0.]
  [15.  0.  0.]
  [15.  0.  1.]
  [13.  0.  3.]
  [ 9.  0.  5.]
  [25.  1.  2.]
  [22.  0.  6.]
  [17.  0.  1.]]]


In [50]:
np.where(matall[0,:,2]==14)

(array([49]),)

In [34]:
check_laptimedata(prepared_laptimedata, check_carno=6)                

after ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
rank:
 [16. 15. 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 15. 15. 13.  9. 25. 22. 17. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 16. 15. 15. 15. 15.
 14. 14. 13. 13. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 14. 22. 21. 21.
 21. 20. 18. 18. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 18. 18.
 16. 16. 12.  9.  5.  3.  2.  2.  2.  2.  2.  2.  2.  2.  2. 11. 18. 18.
 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 16. 16. 16. 16. 15. 14.
 13. 12. 10.  7.  4.  4.  3.  3.  2.  2. 16. 19. 19. 19. 17. 15. 15. 15.
 15. 15. 16. 16. 16. 16. 16. 16. 17. 15. 14. 14. 13. 13. 13. 13. 11. 15.
 15. 15. 15. 15. 15. 14. 14. 12. 11.  7.  6.  4.  4.  2.  2.  1.  1. 11.
 20. 20. 20. 20. 20. 20. 20. 19. 19. 19. 19. 19. 18. 18. 15. 12. 10.  9.
  9.  9.]
pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
leaderPitCnt:
 [ 0.  0.  0

In [ ]:
check_laptimedata(laptime_data, check_carno=12)

### check result of test-addnewfeatures

In [58]:
outdir = 'test/'
outputRoot = outdir
os.makedirs(outdir, exist_ok=True)

_task_dir = f'{outdir}/'

useeid = False
interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')

shift='noshift'
laptimedb = _task_dir + f'gluontsdb-{dataset}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{_test_event}-indy-{year}-{shift}.pickle'
print('Load noshift Laptime Dataset:',laptimedb)
with open(laptimedb, 'rb') as f:
    noshift_data = pickle.load(f, encoding='latin1') 
    
shift='shift'
laptimedb = _task_dir + f'gluontsdb-{dataset}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{_test_event}-indy-{year}-{shift}.pickle'
print('Load New Laptime Dataset:',laptimedb)
with open(laptimedb, 'rb') as f:
    shift_data = pickle.load(f, encoding='latin1') 

Load noshift Laptime Dataset: test//gluontsdb-rank-oracle-noip-noeid-all-all-f1min-t2-rIndy500-2018-indy-2018-noshift.pickle
Load New Laptime Dataset: test//gluontsdb-rank-oracle-noip-noeid-all-all-f1min-t2-rIndy500-2018-indy-2018-shift.pickle


In [59]:
mat49 = print_laptimedata(noshift_data, check_cars = [6], check_laps = [25,35])

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 10, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 16. 16. 15. 15. 13.  9. 25. 22. 17.]
leaderPitCnt:
 [0. 0. 0. 0. 1. 3. 5. 2. 6. 1.]
[[[16.  0.  0.]
  [16.  0.  0.]
  [16.  0.  0.]
  [15.  0.  0.]
  [15.  0.  1.]
  [13.  0.  3.]
  [ 9.  0.  5.]
  [25.  1.  2.]
  [22.  0.  6.]
  [17.  0.  1.]]]


In [60]:
mat49_shift = print_laptimedata(shift_data, check_cars = [6], check_laps = [25,35])

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 10, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 16. 16. 15. 15. 13.  9. 25. 22. 17.]
leaderPitCnt:
 [0. 0. 0. 1. 1. 3. 4. 3. 7. 2.]
[[[16.  0.  0.]
  [16.  0.  0.]
  [16.  0.  0.]
  [15.  0.  1.]
  [15.  0.  1.]
  [13.  0.  3.]
  [ 9.  0.  4.]
  [25.  1.  3.]
  [22.  0.  7.]
  [17.  0.  2.]]]


In [61]:
matall = print_laptimedata(noshift_data, check_cars = [6])

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 200, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 15. 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 15. 15. 13.  9. 25. 22. 17. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 16. 15. 15. 15. 15.
 14. 14. 13. 13. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 14. 22. 21. 21.
 21. 20. 18. 18. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 18. 18.
 16. 16. 12.  9.  5.  3.  2.  2.  2.  2.  2.  2.  2.  2.  2. 11. 18. 18.
 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 16. 16. 16. 16. 15. 14.
 13. 12. 10.  7.  4.  4.  3.  3.  2.  2. 16. 19. 19. 19. 17. 15. 15. 15.
 15. 15. 16. 16. 16. 16. 16. 16. 17. 15. 14. 14. 13. 13. 13. 13. 11. 15.
 15. 15. 15. 15. 15. 14. 14. 12. 11.  7.  6.  4.  4.  2.  2.  1.  1. 11.
 20. 20. 20. 20. 20. 20. 20. 19. 19. 19. 19. 19. 18. 18. 1

In [62]:
matall_shift = print_laptimedata(shift_data, check_cars = [6])

check ====event:Indy500-2018, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 200, 10)
carids: {6: 0}
car: 6  pits:
 (array([ 32,  49,  69, 105, 136, 179, 190]),)
rank:
 [16. 15. 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 15. 15. 13.  9. 25. 22. 17. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 16. 15. 15. 15. 15.
 14. 14. 13. 13. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 14. 22. 21. 21.
 21. 20. 18. 18. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 18. 18.
 16. 16. 12.  9.  5.  3.  2.  2.  2.  2.  2.  2.  2.  2.  2. 11. 18. 18.
 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 17. 16. 16. 16. 16. 15. 14.
 13. 12. 10.  7.  4.  4.  3.  3.  2.  2. 16. 19. 19. 19. 17. 15. 15. 15.
 15. 15. 16. 16. 16. 16. 16. 16. 17. 15. 14. 14. 13. 13. 13. 13. 11. 15.
 15. 15. 15. 15. 15. 14. 14. 12. 11.  7.  6.  4.  4.  2.  2.  1.  1. 11.
 20. 20. 20. 20. 20. 20. 20. 19. 19. 19. 19. 19. 18. 18. 1

In [87]:
laptimedb = _task_dir + f's0ltyple.pickle'
print('Load New Laptime Dataset:',laptimedb)
with open(laptimedb, 'rb') as f:
    c60_data = pickle.load(f, encoding='latin1') 

Load New Laptime Dataset: test//s0ltyple.pickle


In [86]:
_ = print_laptimedata(c60_data, check_cars = [18],check_event='Indy500-2013')

check ====event:Indy500-2013, prediction_len=2,train_len=40, max_len=200, min_len=200,context_len=10
out_mat shape: (1, 200, 10)
carids: {18: 0}
car: 18  pits:
 (array([ 30,  39,  57,  90, 120, 151, 182]),)
rank:
 [24. 24. 24. 24. 22. 22. 22. 22. 22. 22. 22. 22. 21. 21. 22. 22. 22. 22.
 22. 21. 21. 21. 21. 21. 21. 21. 21. 21. 20. 16. 25. 27. 27. 27. 27. 27.
 26. 26. 24. 26. 26. 26. 25. 25. 24. 24. 24. 24. 23. 23. 24. 24. 24. 24.
 24. 24. 23. 25. 24. 24. 24. 25. 25. 25. 25. 26. 26. 26. 26. 26. 26. 26.
 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 26. 25. 25. 25. 24. 23. 14.
 18. 23. 23. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24. 24.
 24. 24. 24. 23. 22. 22. 22. 21. 21. 21. 21. 20. 20. 24. 24. 22. 22. 23.
 23. 23. 23. 23. 23. 23. 23. 23. 22. 22. 21. 23. 23. 23. 23. 23. 21. 21.
 21. 19. 17. 17. 17. 17. 16. 18. 22. 21. 21. 21. 21. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 17. 17. 17. 16. 17. 16. 16. 15. 15. 15. 13. 13. 11.
  9.  7.  8. 18. 18. 18. 18. 18. 18. 18. 17. 17. 17. 16.

In [95]:
print_feature(c60_data, COL_SHIFT_TRACKSTATUS)

FEATURE_STATUS FEATURE_LEADER_PITCNT
print feature: S0L00000
events: 0
car:1
 totallen: 200 feature len: 198
[ 0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.
  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  1.  1.  0.
 nan nan]
car:2
 totallen: 200 feature len: 198
[ 0.  1.  1.  1.  0.  0.  0.  0.  0.  0.

 nan nan]
events: 3
car:2
 totallen: 63 feature len: 61
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0. nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
car:3
 totallen: 200 feature len: 198
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.

 nan nan]
car:32
 totallen: 110 feature len: 108
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.
  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
car:33
 totallen: 46 feature len: 44
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.

In [94]:
%debug

> <ipython-input-89-411a74aa0235>(115)print_feature()
    113             totallen = recx.shape[1]
    114 
--> 115             nans, x= nan_helper(rec[check_feature,:])
    116             nan_count = np.sum(nans)
    117             recz = rec[:, ~np.isnan(rec[check_feature,:])]

ipdb> p rec[check_feature,:]
*** IndexError: index 32 is out of bounds for axis 0 with size 15
ipdb> ｑｕｉｔ
ipdb> quit


In [73]:
print_leaderpitcnt(noshift_data)

events: 0
leaderPitCnt car:1
 totallen: 200 leaderlen: 200
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 10.  2.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  3. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  4.  3.  2.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  6.  2.  1.  4.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  2.  3.  0.  4.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
leaderPitCnt car:2
 totallen: 200 leaderlen: 200
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  

 0. 0. 0. 0. 0. 0. 0. 0.]
leaderPitCnt car:88
 totallen: 62 leaderlen: 62
[ 0.  0.  0.  0.  0.  0.  2.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  6.
  5.  2.  1.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  1.  0. nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
leaderPitCnt car:98
 totallen: 200 leaderlen: 200
[ 0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0

  0.  0.]
leaderPitCnt car:29
 totallen: 200 leaderlen: 200
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  3.  5.  1.  2.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 16.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  3.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  2.  1.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  1.  2.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
leaderPitCnt car:30
 totallen: 46 leaderlen: 46
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  